# Real or Not? NLP with Disaster Tweets
Predict which Tweets are about real disasters and which ones are not

we have files which include a train data,test data and sample submission file ,
We will be predicting whether a given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.
using natural language processing.

here we will split training data into train and test split ,to test our model performance,post that we use our machine learning model to test on unseen data,that is test set.

In [ ]:
import pandas as pd
import numpy  as np

In [ ]:
df=pd.read_csv('../input/nlp-getting-started/train.csv')
df

In [ ]:
df.info()

In [ ]:
df.keyword.value_counts(dropna=False)

In [ ]:
df.location.value_counts(dropna=False)

In [ ]:
df.drop(labels='location',inplace=True,axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df[df['target']==1].isnull().sum()

### as we can see,out of 61 null values,42 null values in keywords are in Taregt=1,that means where the diisaster has happened,so we will replace the 42 null values with mode count,remaining will drop which are not necessary for the analysis

In [ ]:
df['keyword'].fillna(df[df['target']==1].keyword.mode(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df[df['target']==0].isnull().sum()

In [ ]:
df[df['target']==1].isnull().sum()

In [ ]:
df.target.value_counts()

### We can see we have classes which are balanced,and good for analysis

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

## Lets seperate target and independent variables

In [ ]:
X=df.drop('target',axis=1)
y=df['target']

In [ ]:
X

In [ ]:
X.drop(labels='id',inplace=True,axis=1)

In [ ]:
y

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
X.head()

In [ ]:
# Removing punctuations
X=X.iloc[:,0:2]
X.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(2)]
new_Index=[str(i) for i in list1]
X.columns= new_Index
X.head(5)

In [ ]:
# Convertng headlines to lower case
for index in new_Index:
    X[index]=X[index].str.lower()
X.head(1)

In [ ]:
' '.join(str(x) for x in X.iloc[1,0:2])

In [ ]:
tweet = []### Combining both the columns
for row in range(0,len(X.index)):
    tweet.append(' '.join(str(x) for x in X.iloc[row,0:3]))

In [ ]:
tweet[0]

In [ ]:
len(tweet)

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(tweet)):
    review = re.sub('[^a-zA-Z]', ' ', tweet[i])
    review = review.lower()
    review = review.split() 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[0]

In [ ]:
len(corpus)

In [ ]:
## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(corpus)

In [ ]:
countvector

In [ ]:
traindataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(traindataset, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

In [ ]:
predictions = randomclassifier.predict(X_test)
predictions

In [ ]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mnb=MultinomialNB()
mnb.fit(X_train, y_train)

In [ ]:
y_pred = mnb.predict(X_test)
y_pred

In [ ]:
cm=confusion_matrix(y_test,y_pred)
print(cm)
acc=accuracy_score(y_test,y_pred)
print(acc)
rep=classification_report(y_test,y_pred)
print(rep)

In [ ]:
from sklearn.model_selection import cross_val_score
nb_Kfold_accu = cross_val_score(estimator = mnb, X = X_train, y = y_train, cv = 10)
nb_Kfold_accu=nb_Kfold_accu.mean()
print("Accuracy: {:.2f} %".format(nb_Kfold_accu*100))

### We observed both models Multinomial and random forest ,howver random forest gave us the better results compared to the other,with accuracy of 72 %

### Importing TEST Data

In [ ]:
test=pd.read_csv('../input/nlp-getting-started/test.csv')
test

In [ ]:
test.isnull().sum()

In [ ]:
test.drop(labels=['location'],axis=1,inplace=True)

In [ ]:
test.keyword.value_counts()

### we will replace missing values with most number of repeated keywords,that is mode of the list

In [ ]:
test['keyword'].fillna('deluged',inplace=True)

In [ ]:
test.info()

In [ ]:
df2=test.copy()

### similarly cleaning out test data like training data for prediction

In [ ]:
test.drop(labels=['id'],axis=1,inplace=True)

In [ ]:
# Removing punctuations
test=test.iloc[:,0:2]
test.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list2= [j for j in range(2)]
new_Index1=[str(j) for j in list2]
test.columns= new_Index1
test.head(5)

In [ ]:
' '.join(str(x) for x in test.iloc[1,0:2])

In [ ]:
test_tweet = []
for row in range(0,len(test.index)):
    test_tweet.append(' '.join(str(x) for x in test.iloc[row,0:2]))

In [ ]:
test_tweet[1]

In [ ]:
ps = PorterStemmer()
test_corpus = []
for i in range(len(test_tweet)):
    review1 = re.sub('[^a-zA-Z]', ' ',test_tweet[i])
    review1 = review1.lower()
    review1 = review1.split() 
    review1 = [ps.stem(word) for word in review1 if not word in stopwords.words('english')]
    review1 = ' '.join(review1)
    test_corpus.append(review1)

In [ ]:
test_corpus[0]

### Prediction of  test data using the RF model

In [ ]:
test_dataset = countvector.transform(test_corpus)
predict1 = randomclassifier.predict(test_dataset)
predict1

In [ ]:
sub= pd.DataFrame({'id':df2.id,
                      'target':predict1})

sub.to_csv("outcome.csv",index=False,header = True)